In [4]:
import os
import pandas as pd
import requests
import json

In [5]:
INATURALIST_API_KEY = os.getenv("INATURALIST_API_KEY", "")

### Fetch observations of kikuyugrass in iNaturalist

In [6]:
# fetch only verifiable observations with locations and images
url = "https://api.inaturalist.org/v1/observations"
params = {
    "verifiable": True,
    "photos": True,
    "geo": True,
    "geoprivacy": "open",
    "place_id": 14,  # California
    "taxon_id": 60298,  # Cenchrus clandestinus
    "per_page": 10000,  # max
}
headers = {"Authorization": INATURALIST_API_KEY}

In [7]:
# fetch a single page of observations
def fetch_observations(page):
    params["page"] = page
    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status()
    return response.json()["results"]


# fetch all pages
all_observations = []
page = 1

while True:
    observations = fetch_observations(page)
    if not observations:
        break
    all_observations.extend(observations)
    page += 1

In [8]:
# print the first and last few lines
print("\n".join(json.dumps(all_observations[0], indent=2).split("\n")[:40]))
print("...................................................................")
print("\n".join(json.dumps(all_observations[0], indent=2).split("\n")[-40:]))

{
  "quality_grade": "needs_id",
  "time_observed_at": "2024-07-18T11:09:00-07:00",
  "taxon_geoprivacy": "open",
  "annotations": [],
  "uuid": "cf6f9a84-0996-4ded-b2e0-a7bd12d67217",
  "observed_on_details": {
    "date": "2024-07-18",
    "day": 18,
    "month": 7,
    "year": 2024,
    "hour": 11,
    "week": 29
  },
  "id": 230179086,
  "cached_votes_total": 0,
  "identifications_most_agree": false,
  "created_at_details": {
    "date": "2024-07-18",
    "day": 18,
    "month": 7,
    "year": 2024,
    "hour": 14,
    "week": 29
  },
  "species_guess": "Kikuyu Grass",
  "identifications_most_disagree": false,
  "tags": [],
  "positional_accuracy": 4,
  "comments_count": 1,
  "site_id": 1,
  "created_time_zone": "America/Los_Angeles",
  "license_code": null,
  "observed_time_zone": "America/Los_Angeles",
  "quality_metrics": [],
  "public_positional_accuracy": 4,
  "reviewed_by": [
    497764
  ],
  "oauth_application_id": null,
.....................................................

### Extract desired fields

In [9]:
def extract_observation_data(observation):
    observed_on_details = observation.get("observed_on_details") or {}
    taxon = observation.get("taxon") or {}

    data = {
        "quality_grade": observation.get("quality_grade"),
        "uuid": observation.get("uuid"),
        "observed_on_date": observation.get("observed_on"),
        "observed_on_day": observed_on_details.get("day"),
        "observed_on_month": observed_on_details.get("month"),
        "observed_on_year": observed_on_details.get("year"),
        "id": observation.get("id"),
        "positional_accuracy": observation.get("positional_accuracy"),
        "public_positional_accuracy": observation.get("public_positional_accuracy"),
        "description": observation.get("description"),
        "captive": observation.get("captive"),
        "uri": observation.get("uri"),
        "geojson": observation.get("geojson"),
        "location": observation.get("location"),
        "place_guess": observation.get("place_guess"),
        "taxon_name": taxon.get("name"),
        "taxon_min_species_taxon_id": taxon.get("min_species_taxon_id"),
        "preferred_common_name": taxon.get("preferred_common_name"),
    }

    observation_photos = observation.get("observation_photos", [])
    if observation_photos:
        first_photo_details = observation_photos[0].get("photo") or {}
        data["observation_photo_license_code"] = first_photo_details.get("license_code")

    for idx, photo in enumerate(observation_photos):
        data[f"observation_photo_{idx}_id"] = photo.get("id")
        photo_details = photo.get("photo") or {}
        data[f"observation_photo_{idx}_original_dimensions"] = photo_details.get(
            "original_dimensions"
        )
        data[f"observation_photo_{idx}_url"] = photo_details.get("url")

    return data


filtered_data = [extract_observation_data(obs) for obs in all_observations]
kikuyu_data = pd.DataFrame(filtered_data)

In [10]:
pd.set_option("display.max_columns", None)
kikuyu_data.head()

,quality_grade,uuid,observed_on_date,observed_on_day,observed_on_month,observed_on_year,id,positional_accuracy,public_positional_accuracy,description,captive,uri,geojson,location,place_guess,taxon_name,taxon_min_species_taxon_id,preferred_common_name,observation_photo_license_code,observation_photo_0_id,observation_photo_0_original_dimensions,observation_photo_0_url,observation_photo_1_id,observation_photo_1_original_dimensions,observation_photo_1_url,observation_photo_2_id,observation_photo_2_original_dimensions,observation_photo_2_url,observation_photo_3_id,observation_photo_3_original_dimensions,observation_photo_3_url,observation_photo_4_id,observation_photo_4_original_dimensions,observation_photo_4_url,observation_photo_5_id,observation_photo_5_original_dimensions,observation_photo_5_url,observation_photo_6_id,observation_photo_6_original_dimensions,observation_photo_6_url,observation_photo_7_id,observation_photo_7_original_dimensions,observation_photo_7_url,observation_photo_8_id,observation_photo_8_original_dimensions,observation_photo_8_url,observation_photo_9_id,observation_photo_9_original_dimensions,observation_photo_9_url
0,needs_id,cf6f9a84-0996-4ded-b2e0-a7bd12d67217,2024-07-18,18.0,7.0,2024.0,230179086,4.0,4.0,Temescal Canyon Park,False,https://www.inaturalist.org/observations/23017...,"{'type': 'Point', 'coordinates': [-118.5320907...","34.0412872581,-118.5320907202","Pacific Palisades, Los Angeles, CA, USA",Cenchrus clandestinus,60298,Kikuyu Grass,None,380635383.0,"{'width': 2048, 'height': 1412}",https://static.inaturalist.org/photos/40866931...,380635384.0,"{'width': 2048, 'height': 1336}",https://static.inaturalist.org/photos/40866929...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,needs_id,39f544ee-5509-49ea-a296-19c825217ee1,2024-07-17,17.0,7.0,2024.0,230006077,35.0,35.0,,False,https://www.inaturalist.org/observations/23000...,"{'type': 'Point', 'coordinates': [-117.0604807...","32.6589614448,-117.0604807898","Fairlomas Rd, National City, CA, US",Cenchrus clandestinus,60298,Kikuyu Grass,None,380333087.0,"{'width': 1536, 'height': 2048}",https://static.inaturalist.org/photos/40834622...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,needs_id,28751c0a-3cd3-4f64-9058-75e564f328f8,2024-07-15,15.0,7.0,2024.0,229533712,68.0,68.0,None,False,https://www.inaturalist.org/observations/22953...,"{'type': 'Point', 'coordinates': [-120.8232040...","35.3221435547,-120.8232040405",Baywood-Los Osos,Cenchrus clandestinus,60298,Kikuyu Grass,cc-by-nc,379495624.0,"{'width': 1536, 'height': 2048}",https://inaturalist-open-data.s3.amazonaws.com...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,research,03159809-00bc-4852-bbd6-2aab8038a6ba,2024-06-24,24.0,6.0,2024.0,229078028,4.0,4.0,,False,https://www.inaturalist.org/observations/22907...,"{'type': 'Point', 'coordinates': [-122.534875,...","37.83163,-122.534875","Marin Headlands, Sausalito, CA, US",Cenchrus clandestinus,60298,Kikuyu Grass,cc-by-nc,378698003.0,"{'width': 1536, 'height': 2048}",https://inaturalist-open-data.s3.amazonaws.com...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,needs_id,e73c596e-d6d4-4980-afaf-be9268e70964,2024-07-10,10.0,7.0,2024.0,228696199,NaN,NaN,None,False,https://www.inaturalist.org/observations/22869...,"{'type': 'Point', 'coordinates': [-121.8467233...","36.9169416667,-121.8467233333","Manresa State Beach, Santa Cruz, California, U...",Cenchrus clandestinus,60298,Kikuyu Grass,cc-by-nc,378022596.0,"{'width': 1536, 'height': 2048}",https://inaturalist-open-data.s3.amazonaws.com...,378022597.0,"{'width': 2048, 'height': 2048}",https://inaturalist-open-data.s3.amazonaws.com...,378022598.0,"{'width': 2048, 'height': 1536}",https://inaturalist-open-data.s3.amazonaws.com...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [11]:
# for image urls, replace  “square” with “small”, “medium”, “large”, or “original” to get other sizes
# https://forum.inaturalist.org/t/where-to-get-high-quality-images-for-observations-from-the-api/48134/2

### Save csv

In [12]:
kikuyu_data.to_csv("./inaturalist_observations.csv")